In [1]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

In [2]:
# Load the historical data
df = pd.read_csv('youTube_video_process_data.csv')

In [3]:
df.head()

,title,view_count,like_count,comment_count,favorite_count,durations,published_at,published_date,published_time
0,Full Energy New Day With Starbucks Coffee Musi...,1351,11,0,0,P0D,2023-10-05 22:01:46+00:00,2023-10-05,22:01:46
1,Starbucks Happy Morning Music - Relaxing Starb...,606,11,4,0,P1D,2023-10-05 22:00:11+00:00,2023-10-05,22:00:11
2,Starbucks Coffee Music & Jazz Relaxing Music -...,1649,21,2,0,PT11H54M57S,2023-10-05 13:28:00+00:00,2023-10-05,13:28:00
3,Starbucks Coffee Shop Music - Keep Upbeat Your...,357,3,0,0,P0D,2023-10-05 08:53:58+00:00,2023-10-05,08:53:58
4,Elegant Starbucks Cafe Music - Starbucks Music...,1847,22,6,0,P1DT1S,2023-10-04 22:00:07+00:00,2023-10-04,22:00:07


### Data Cleaning and Preprocessing

In [8]:
# Assuming 'created' contains Unix timestamps (seconds since epoch)
df['published_date'] = pd.to_datetime(df['published_date'])

#### Remove Unnecessary Columns

In [10]:
df = df.drop(['title', 'like_count', 'comment_count', 'favorite_count', 'durations', 'published_at', 'published_time'], axis=1)

In [11]:
df.head()

,view_count,published_date
0,1351,2023-10-05
1,606,2023-10-05
2,1649,2023-10-05
3,357,2023-10-05
4,1847,2023-10-04


In [12]:
# Sort the data by the 'created' column and split it into training and testing sets.
df.sort_values(by='published_date', inplace=True, ascending=False)

In [13]:
df.set_index('published_date', inplace=True)

In [14]:
df

,view_count
published_date,
2023-10-05,1351
2023-10-05,1357
2023-10-05,1649
2023-10-05,357
2023-10-05,1357
...,...
2023-06-20,3504
2023-06-20,3504
2023-06-20,3504


In [15]:
# Split the data into training and testing sets with test_set containing the newest data
train_set, test_set = df[int(len(df) * 0.2):], df[:int(len(df) * 0.2)]

#### Model Training:

In [16]:
# Define the order of the ARIMA model (p, d, q)
p = 1  # Autoregressive order
d = 1  # Differencing order (to make the series stationary)
q = 1  # Moving average order

In [18]:
# Create the ARIMA model using the 'view_count' column
arima_model = sm.tsa.ARIMA(train_set['view_count'], order=(p, d, q))

/usr/local/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/site-packages/statsm

In [19]:
# Fit the ARIMA model to your training data
arima_model_fit = arima_model.fit()

In [20]:
# Print the summary of the model (optional)
print(arima_model_fit.summary())

                               SARIMAX Results                                
Dep. Variable:             view_count   No. Observations:                  800
Model:                 ARIMA(1, 1, 1)   Log Likelihood               -6980.519
Date:                Fri, 06 Oct 2023   AIC                          13967.038
Time:                        15:44:22   BIC                          13981.088
Sample:                             0   HQIC                         13972.436
                                - 800                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.0291      0.017     -1.683      0.092      -0.063       0.005
ma.L1         -0.8002      0.013    -63.719      0.000      -0.825      -0.776
sigma2      2.268e+06    5.7e+04     39.765      0.0

#### Model Evaluation:

In [21]:
# Make predictions on the testing data
predictions = arima_model_fit.predict(start=0, end=len(test_set) - 1, typ='levels')

In [22]:
# Calculate evaluation metrics
mae = mean_absolute_error(test_set, predictions)
mse = mean_squared_error(test_set, predictions)
rmse = np.sqrt(mse)

NameError: name 'mean_absolute_error' is not defined

#### Forecasting:

In [23]:
# Define the number of periods you want to forecast (e.g., one week)
forecast_periods = 7  # Adjust as needed

In [24]:
# Use the trained ARIMA model to make forecasts
forecast = arima_model_fit.forecast(steps=forecast_periods)

/usr/local/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [25]:
last_date = df.index[0]

In [26]:
last_date

Timestamp('2023-10-05 00:00:00')

In [27]:
last_date = pd.to_datetime(last_date)  # Assuming 'last_date' is a string

In [28]:
forecast_dates = pd.date_range(start=last_date + pd.DateOffset(1), periods=forecast_periods)

In [29]:
# Create a DataFrame to store the forecasted values and dates
forecast_df = pd.DataFrame({'Date': forecast_dates, 'Forecast': forecast})

In [30]:
# Print the forecasted values for the next week
print(forecast_df)

          Date     Forecast
800 2023-10-06  3132.685227
801 2023-10-07  3143.475731
802 2023-10-08  3143.162156
803 2023-10-09  3143.171269
804 2023-10-10  3143.171004
805 2023-10-11  3143.171011
806 2023-10-12  3143.171011


In [32]:
# Specify the file path where you want to save the model
model_save_path = 'arima_youtube_view_model.pkl'

In [33]:
# Save the ARIMA model to the specified file
arima_model_fit.save(model_save_path)